In [1]:
import functools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torchvision import datasets, transforms
from collections import deque
#from ULogicalModels import UDecisionTreeClassifier
#from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, ConfusionMatrixDisplay, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tqdm as tqdm

In [37]:
def getModelError(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    y_predict = model.predict(X_test)
    return classification_report(y_test, y_predict)
    
def compareModels(model, uModel, X, y, is_need_scale):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)
    if is_need_scale:
        scaler = StandardScaler()
        scaler.fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)
    model_error = getModelError(model, X_train, X_test, y_train, y_test)
    uModel_error = getModelError(uModel, X_train, X_test, y_train, y_test)
    print('Ошибка на пакетной модели\n', model_error)
    print('Ошибка на реализованной модели\n', uModel_error)

In [38]:
#Вины
data_1 = pd.read_csv('DATA/wine_fraud.csv')
X = data_1.drop(['type','quality'], axis=1)
y = data_1['type']
y = y.map({'red': 1, 'white': -1})

In [8]:
#Исследование слуха
data_1 = pd.read_csv('DATA/hearing_test.csv')
X = data_1.drop('test_result', axis=1)
y = data_1['test_result']

In [2]:
#Ирисы
data_1 = pd.read_csv('DATA/iris.csv')
X = data_1.drop('species', axis=1)
y = data_1['species']

In [35]:
model = AdaBoostClassifier(n_estimators=10)
uModel = UAdaBoost()

In [39]:
compareModels(model, uModel, X, y, False)

Ошибка на пакетной модели
               precision    recall  f1-score   support

          -1       0.99      0.99      0.99      1451
           1       0.98      0.97      0.97       499

    accuracy                           0.99      1950
   macro avg       0.98      0.98      0.98      1950
weighted avg       0.99      0.99      0.99      1950

Ошибка на реализованной модели
               precision    recall  f1-score   support

          -1       0.99      0.99      0.99      1451
           1       0.97      0.98      0.97       499

    accuracy                           0.99      1950
   macro avg       0.98      0.98      0.98      1950
weighted avg       0.99      0.99      0.99      1950



C:\Users\SMD\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


In [2]:
class Perceptron(torch.nn.Module):
    def __init__(self, 
                 input_dim=11, 
                 num_layers=0,
                 hiden_dim=30, 
                 output_dim=3,
                 p=0.0,
                 device='cpu'):
        super(Perceptron, self).__init__()
        self.layers = torch.nn.Sequential()
        prev_size = input_dim
        for i in range(num_layers):
            self.layers.add_module(f'layer{i+1}',
                                   torch.nn.Linear(prev_size, hiden_dim)) #Шаблон вычисления слоя
            self.layers.add_module(f'tanh{i+1}',
                                   torch.nn.Tanh()) #Сигма функция(функция активации)
            self.layers.add_module(f'dropout{i+1}',
                                   torch.nn.Dropout(p=p)) #Удаление нейрона с вероятностью p
            prev_size = hiden_dim
        self.layers.add_module('classifier',
                               torch.nn.Linear(prev_size, output_dim)) #Выход сети
        self.to(device)

    def forward(self, input):
        return self.layers(input)

In [2]:
#Обучение модели
def trainer(model, X, y, loss_function, optimizer, epochs):
    for epoch in range(epochs): #Кол-во итераций
        optimizer.zero_grad()
        X = X.to(device)
        y = y.to(device)

        output = model(X)
        loss = loss_function(output, y)
        loss.backward()
        optimizer.step()

#Тестрирование модели
def testing(model, X):    
    X = X.to(device)
    y_pred = torch.argmax(model(X), dim=1)
    return y_pred

#Преобразуем в Tensor
def getTensor(X, y):
    X_tensor = torch.from_numpy(X.values.astype(np.float32))
    y_tensor = torch.from_numpy(y.values.astype(np.int64))
    return X_tensor, y_tensor

In [5]:
#Обучение

In [5]:
#Устройство, на котором будут выполняться вычисления
device = torch.device('cpu')

In [3]:
#Разбиение выборки на Train и Test
data_1 = pd.read_csv('DATA/wine_fraud.csv')
X = data_1.drop(['type','quality'], axis=1)
y = data_1['type']
y = y.map({'red': 2, 'white': 0})

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)
X_train, y_train = getTensor(X_train, y_train)
X_test, y_test = getTensor(X_test, y_test)

In [39]:
X_train.shape, X_test.shape

(torch.Size([5197, 11]), torch.Size([1300, 11]))

In [52]:
#Создание модели
model = Perceptron(num_layers=5, device=device)

In [53]:
#Обучение
_ = model.train()
#trainer(model=model,
#        X=X_train,
#        y=y_train,
#        loss_function=torch.nn.CrossEntropyLoss(),
#        optimizer=torch.optim.Adam(model.parameters(), lr=0.001),
#        epochs=200)

In [54]:
_ = model.eval()
y_pred = testing(model, X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       971
           1       0.00      0.00      0.00         0
           2       0.74      0.41      0.53       329

    accuracy                           0.10      1300
   macro avg       0.25      0.14      0.18      1300
weighted avg       0.19      0.10      0.13      1300



C:\Users\SMD\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\SMD\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\SMD\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{met

In [51]:
model

Perceptron(
  (layers): Sequential(
    (classifier): Linear(in_features=11, out_features=3, bias=True)
  )
)

In [47]:
#Сверточные Нейронные сети

In [2]:
def train_on_batch(model, x_batch, y_batch, optimizer, loss_function):
    model.train()
    model.zero_grad()

    output = model(x_batch.to('cpu'))

    loss = loss_function(output, y_batch.to('cpu'))
    loss.backward()

    optimizer.step()
    return loss.cpu().item()   

In [3]:
def train_epoch(train_generator, model, loss_function, optimizer, callback=None):
    epoch_loss = 0
    total = 0
    for it, (batch_of_x, batch_of_y) in enumerate(train_generator):
        batch_loss = train_on_batch(model, batch_of_x.to('cpu'), batch_of_y.to('cpu'), optimizer, loss_function)

        if callback is not None:
            callback(model, batch_loss)

        epoch_loss += batch_loss * len(batch_of_x)
        total += len(batch_of_x)

    return epoch_loss / total

In [4]:
def trainer(count_of_epoch,
            batch_size,
            dataset,
            model,
            loss_function,
            optimizer,
            lr=0.001,
            callback=None):
    optima = optimizer(model.parameters(), lr=lr)
    for it in range(count_of_epoch):
        batch_generator = torch.utils.data.DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True)
        epoch_loss = train_epoch(train_generator=batch_generator,
                                 model=model,
                                 loss_function=loss_function,
                                 optimizer=optima,
                                 callback=callback)
        print(f'Эпоха {it}: ошибка = {epoch_loss}') 

In [5]:
def tester(model, dataset, batch_size):
    batch_generator = torch.utils.data.DataLoader(dataset=dataset, batch_size=batch_size)
    y_predict = []
    y_real = []
    for it, (x_batch, y_batch) in enumerate(batch_generator):
        x_batch = x_batch.to('cpu')
        y_batch = y_batch.to('cpu')

        output = model(x_batch)
        y_predict.extend(torch.argmax(output, dim=-1).cpu().numpy().tolist())
        y_real.extend(y_batch.cpu().numpy().tolist())
    print(classification_report(y_real, y_predict))    

In [6]:
class CNN(torch.nn.Module):
    @property
    def device(self):
        for param in self.parameters():
            return param.device

    def __init__(self):
        super(CNN, self).__init__()
        self.layers = torch.nn.Sequential()
        self.layers.add_module('conv1', torch.nn.Conv2d(1, 1*6, kernel_size=5)) #Кол-во входных каналов(признаков, которые 
                                                                                #описывают объект)
                                                                                #Кол-во выходных каналов(признаков) 
                                                                                #Размер сверточного ядра(Оно сжимает изображение)
        self.layers.add_module('relu1', torch.nn.ReLU()) #Функция активации
        self.layers.add_module('pool1', torch.nn.MaxPool2d(kernel_size=2)) #Сжимает изображение ещё в 2 раза
        self.layers.add_module('conv2', torch.nn.Conv2d(1*6, 1*16, kernel_size=5))
        self.layers.add_module('relu2', torch.nn.ReLU())
        self.layers.add_module('pool2', torch.nn.MaxPool2d(kernel_size=2))
        self.layers.add_module('flatten', torch.nn.Flatten()) #Вытягиваем тензор в один вектор
        self.layers.add_module('linear1', torch.nn.Linear(16*4*4, 120))
        self.layers.add_module('relu3', torch.nn.ReLU())
        self.layers.add_module('linear2', torch.nn.Linear(120, 84))
        self.layers.add_module('relu4', torch.nn.ReLU())
        self.layers.add_module('linear3', torch.nn.Linear(84, 10))

    def forward(self, input):
        return self.layers(input)

In [ ]:
#Обучение модели CNN

In [7]:
#Загрузка данных
MNIST_train = datasets.MNIST('DATA/mnist', train=True, download=True, transform=transforms.ToTensor())
MNIST_test = datasets.MNIST('DATA/mnist', train=False, download=True, transform=transforms.ToTensor())

In [8]:
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam

model = CNN()
model.to('cpu')

CNN(
  (layers): Sequential(
    (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (relu1): ReLU()
    (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (relu2): ReLU()
    (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (flatten): Flatten(start_dim=1, end_dim=-1)
    (linear1): Linear(in_features=256, out_features=120, bias=True)
    (relu3): ReLU()
    (linear2): Linear(in_features=120, out_features=84, bias=True)
    (relu4): ReLU()
    (linear3): Linear(in_features=84, out_features=10, bias=True)
  )
)

In [9]:
#Обучение CNN
trainer(count_of_epoch=1,
        batch_size=64,
        dataset=MNIST_train,
        model=model,
        loss_function=loss_function,
        optimizer=optimizer,
        lr=0.001,
        callback=None) #callback - позволяет отслеживать обучение модели(логирование). Используется TensorBoard

Эпоха 0: ошибка = 0.33032608373363814


In [10]:
tester(model=model,
       dataset=MNIST_test,
       batch_size=64)

              precision    recall  f1-score   support

           0       0.97      0.98      0.98       980
           1       0.99      0.97      0.98      1135
           2       0.98      0.96      0.97      1032
           3       0.98      0.93      0.96      1010
           4       0.99      0.95      0.97       982
           5       0.95      0.97      0.96       892
           6       0.97      0.99      0.98       958
           7       0.97      0.96      0.96      1028
           8       0.91      0.98      0.94       974
           9       0.94      0.97      0.95      1009

    accuracy                           0.96     10000
   macro avg       0.96      0.96      0.96     10000
weighted avg       0.97      0.96      0.96     10000



In [31]:
#Выбрали случайно объект
x = data[torch.randint(data.shape[0], size=(1,))].reshape(-1,)
#Выходы нейронов
neuron_outputs = 

In [38]:
x.reshape(-1, )

tensor([6.8000e+00, 1.8000e-01, 2.8000e-01, 1.1000e+00, 2.7000e-02, 3.2000e+01,
        1.1200e+02, 9.9089e-01, 3.1500e+00, 4.5000e-01, 1.1000e+01])

In [71]:
weigths = torch.Tensor([[[-4,5,7],
                         [2,-3,1]],
                        [[9,2,3],
                         [1,3,1]]
                       ])
x = torch.Tensor([2,-1,-2])

In [60]:
a = weigths[0] @ x

In [61]:
a

tensor([-27.,   5.])

In [ ]:
input_layer = x
hiden_layers
output_layer = Linear()

In [62]:
class LinearLayer:
    def __init__(self,
                 in_features,
                 out_features,
                 activation_function='ReLU',
                 start_weights='random'):
        self.activation_function = activation_function
        self.weights_ = self._start_weights_type[start_weights](in_features, out_features)

    def __repr__(self):
        return (f'Linear(in_dim={self.weights_.shape[0]}, '
                f'out_dim={self.weights_.shape[1]}, '
                f'activation_F={self.activation_function})')

    def findNeuronsOutput(self, x):
        return self.weights_.T @ x
    
    def findLayerOutput(self, neurons_out):
        return self._activation_function_type[self.activation_function](neurons_out)

    def findLayerDerivativeOutput(self, neurons_out):
        return self._activation_function_dir_type[self.activation_function](neurons_out)

    @staticmethod
    def _zeroStartWeights(rows, cols):
        return torch.zeros((rows, cols)).type(torch.float64)

    @staticmethod
    def _randomStartWeights(rows, cols):
        return torch.rand((rows, cols)).type(torch.float64)

    @staticmethod
    def _linActivationF(y):
        return y.type(torch.float64)
    
    @staticmethod
    def _reLUActivationF(y):
        return torch.where(y>=0, y, 0).type(torch.float64)

    @staticmethod
    def _dirLinActivationF(y):
        return torch.ones(y.shape[0]).type(torch.float64)

    @staticmethod
    def _dirReLUActivationF(y):
        return torch.where(y>=0, 1, 0).type(torch.float64)

    _start_weights_type = {'zeros': _zeroStartWeights,
                           'random': _randomStartWeights}
    _activation_function_type = {'linear': _linActivationF,
                                 'ReLU': _reLUActivationF}
    _activation_function_dir_type = {'linear': _dirLinActivationF,
                                     'ReLU': _dirReLUActivationF}

In [93]:
#МОЖЕТ ХРАНИТЬ ВСЕ ВЫХОДЫ СЛОЯ В СЛОЕ
class LayerSequential:
    def __init__(self):
        self.layers_name = []
        self.layers = []

    def add_module(self, layer_name, layer):
        self.layers_name.append(layer_name)
        self.layers.append(layer)

    def forward(self, x):
        layers_outs, layers_dir_outs = [], []
        for layer in self.layers:
            neurons_output = layer.findNeuronsOutput(x)
            layer_output = layer.findLayerOutput(neurons_output)
            layer_dir_output = layer.findLayerDerivativeOutput(neurons_output)
            layers_outs.append(layer_output)
            layers_dir_outs.append(layer_dir_output)
            x = layer_output
        return (self._getTensor(layers_outs),
                self._getTensor(layers_dir_outs))

    def backward(self, layers_outs, layers_dir_outs, x, y, loss_function='square'):
        errors = self._loss_function_dir_type[loss_function](layers_outs[-1], y).reshape(1,-1)
        layers_weights = list(layer.weights_ for layer in self.layers)
        for weights, layer_dir in zip(layers_weights[::-1], torch.flip(layers_dir_outs[1:], dims=[0])):
            layer_error = (weights @ (errors[0] * layer_dir)).reshape(1,-1)
            errors = torch.cat([layer_error,errors])
        
        for index, (weights, error, layer_dir, layer_out) in enumerate(zip(layers_weights,
                                                                           errors,
                                                                           layers_dir_outs,
                                                                           torch.roll(layers_outs, 1, 0))):
            
            print(index)
            print(weights)
            print(error)
            print(layer_dir)
            print(layer_out)
            

    @staticmethod
    def _getTensor(array):
        return torch.cat([x.reshape(1,-1) for x in array]) 

    @staticmethod
    def _dirSquareError(x, y):
        return (x - y).type(torch.float64)
    
    _loss_function_dir_type = {'square': _dirSquareError}

In [95]:
x = torch.Tensor([2,3,4,2]).type(torch.float64)
y = torch.Tensor([1,1,1]).type(torch.float64)
layers = LayerSequential()
layers.add_module('Linear1', LinearLayer(4,3))
layers.add_module('Linear2', LinearLayer(3,3))
layers.add_module('Linear3', LinearLayer(3,3))
layers.add_module('Linear4', LinearLayer(3,3))
layers_outs, layers_dir_outs = layers.forward(x)
layers.backward(layers_outs, layers_dir_outs, x, y)

0
tensor([[0.1035, 0.0193, 0.8211],
        [0.1392, 0.8908, 0.5626],
        [0.5488, 0.3482, 0.4688],
        [0.2668, 0.0305, 0.6823]], dtype=torch.float64)
tensor([53.9990, 14.1581, 64.4954], dtype=torch.float64)
tensor([1., 1., 1.], dtype=torch.float64)
tensor([3.3533, 4.1650, 6.5698], dtype=torch.float64)
1
tensor([[0.6114, 0.1829, 0.9269],
        [0.3190, 0.0971, 0.0210],
        [0.7845, 0.6657, 0.4578]], dtype=torch.float64)
tensor([30.0246, 40.5639, 30.4494], dtype=torch.float64)
tensor([1., 1., 1.], dtype=torch.float64)
tensor([8.5328, 5.3911, 6.2035], dtype=torch.float64)
2
tensor([[0.8672, 0.4053, 0.4849],
        [0.6829, 0.8926, 0.4924],
        [0.7565, 0.0333, 0.9303]], dtype=torch.float64)
tensor([ 9.5283, 24.8650, 24.0942], dtype=torch.float64)
tensor([1., 1., 1.], dtype=torch.float64)
tensor([15.7735,  8.4764, 12.5634], dtype=torch.float64)
3
tensor([[0.1082, 0.4144, 0.1250],
        [0.3305, 0.6105, 0.7159],
        [0.4146, 0.3749, 0.8372]], dtype=torch.float64)


In [26]:
for layer_weights, layer_dir in zip(reversed(layer.weights_ for layer in layers.layers), layers_dir_outs[::-1]):
    print(layer_weights, layer_dir)

TypeError: 'generator' object is not reversible

In [97]:
torch.roll(layers_outs, 1, 0)

tensor([[ 9.7175, 16.4220, 18.5567],
        [ 3.3533,  4.1650,  6.5698],
        [ 8.5328,  5.3911,  6.2035],
        [15.7735,  8.4764, 12.5634]], dtype=torch.float64)

In [35]:
[layer.weights_ for layer in layers.layers][::-1]

[tensor([[0.7124, 0.9020, 0.1245],
         [0.8411, 0.1967, 0.1525],
         [0.5350, 0.6078, 0.9847]]),
 tensor([[0.9152, 0.7722, 0.2059],
         [0.7328, 0.4468, 0.8606],
         [0.1081, 0.6390, 0.8466]]),
 tensor([[0.2375, 0.6820, 0.3395],
         [0.7870, 0.7744, 0.4389],
         [0.1797, 0.8624, 0.8896]]),
 tensor([[0.1215, 0.9102, 0.6717],
         [0.0553, 0.3966, 0.5961],
         [0.9424, 0.1465, 0.6675],
         [0.3407, 0.9053, 0.1807]])]

In [27]:
weights = list(layer.weights_ for layer in layers.layers[1:])

In [61]:
torch.ones(3).type(torch.float64)

tensor([1., 1., 1.], dtype=torch.float64)

In [60]:
torch.Tensor([1,2,3,2]).type(torch.float64)

tensor([1., 2., 3., 2.], dtype=torch.float64)